In [3]:
import numpy as np
import pandas as pd

In [4]:
data=pd.read_csv(r"E:\Datasets\brain_stroke.csv")
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


In [5]:
data.isna().sum()

gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [6]:
data.stroke.value_counts()

0    4733
1     248
Name: stroke, dtype: int64

In [7]:
x=data.iloc[:,[1,2,3,7,8]]
y=data.iloc[:,-1]


In [8]:
# Data Imbalancing
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
ro=RandomOverSampler()
x_resample,y_resample=ro.fit_resample(x,y)
print(Counter(y_resample))

Counter({1: 4733, 0: 4733})


In [57]:
# Feature Scaling by StandardScaler()
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
x_data=ss.fit_transform(x_resample)
x_data

array([[ 0.54041876, -0.4594621 ,  2.78363318,  1.95125923,  1.13229791],
       [ 1.12620824, -0.4594621 ,  2.78363318, -0.24569436,  0.4895363 ],
       [-0.27067436, -0.4594621 , -0.35924274,  0.92301981,  0.78740144],
       ...,
       [ 0.27005439, -0.4594621 , -0.35924274,  1.47740298,  0.34844229],
       [ 0.99102605, -0.4594621 , -0.35924274, -0.25822077,  0.25437962],
       [ 1.03608678, -0.4594621 , -0.35924274, -1.09301661, -0.81166403]])

In [63]:
y_data=y_resample

In [64]:
# slpit data into train and test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.2,random_state=100)


In [65]:
print(Counter(y_train))
print(Counter(y_test))

Counter({0: 3792, 1: 3780})
Counter({1: 953, 0: 941})


In [66]:
# Model Imbalancing--> Cross_validation
from sklearn.model_selection import KFold
kf=KFold(n_splits=5,random_state=100,shuffle=True)
kf.get_n_splits(x_data)
print(kf)

KFold(n_splits=5, random_state=100, shuffle=True)


In [67]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
l1=LogisticRegression()

In [68]:
# Without Iteration based cross validation
scores=cross_val_score(l1,x_train,y_train,cv=kf)
y_pred=cross_val_predict(l1,x_test,y_test,cv=kf)
print(scores)
print(y_pred)

[0.77227723 0.75841584 0.77146631 0.78996037 0.79326288]
[0 1 1 ... 1 0 1]


In [108]:
kx_train_data=[]
kx_test_data=[]
ky_train_data=[]
ky_test_data=[]
for train_data,test_data in kf.split(x_data):
    x_train,x_test=x_data[train_data],x_data[test_data]
    y_train,y_test=y_data[train_data],y_data[test_data]
    scores=cross_val_score(l1,x_train,y_train,cv=kf)
    y_pred=cross_val_predict(l1,x_test,y_test,cv=kf)
    kx_train_data.append(x_train)
    kx_test_data.append(x_test)
    ky_train_data.append(y_train)
    ky_test_data.append(y_test)
    print("Checking Balancing y test.......:",Counter(y_test))
    print(y_pred)
    print(scores)
    print(np.mean(scores)*100)

    

Checking Balancing y test.......: Counter({1: 953, 0: 941})
[1 0 0 ... 1 1 1]
[0.77359736 0.77821782 0.77410832 0.76948481 0.78665786]
77.64132344542249
Checking Balancing y test.......: Counter({0: 969, 1: 924})
[1 1 1 ... 0 1 1]
[0.78085809 0.77161716 0.76963696 0.77212682 0.77939234]
77.47262731557171
Checking Balancing y test.......: Counter({1: 957, 0: 936})
[1 1 1 ... 1 1 1]
[0.78613861 0.76171617 0.77821782 0.76882431 0.77939234]
77.4857850382132
Checking Balancing y test.......: Counter({0: 947, 1: 946})
[1 1 1 ... 0 1 1]
[0.77029703 0.78415842 0.77887789 0.78005284 0.75495376]
77.36679876706297
Checking Balancing y test.......: Counter({1: 953, 0: 940})
[0 1 1 ... 1 1 1]
[0.76435644 0.78019802 0.78745875 0.7661823  0.78467635]
77.6574370779218


In [114]:
for train_data,test_data in kf.split(x_data,y_data):
    x_train,x_test=x_data[train_data],x_data[test_data]
    y_train,y_test=y_data[train_data],y_data[test_data]
    scores=cross_val_score(l1,x_train,y_train,cv=kf)
    y_pred=cross_val_predict(l1,x_test,y_test,cv=kf)
    print("Checking y_test Balancing:",Counter(y_test))
    print("Checking y_train Balancing:",Counter(y_train))
    print(scores)
    print("----------------------------------------------------------")

Checking y_test Balancing: Counter({1: 953, 0: 941})
Checking y_train Balancing: Counter({0: 3792, 1: 3780})
[0.77359736 0.77821782 0.77410832 0.76948481 0.78665786]
----------------------------------------------------------
Checking y_test Balancing: Counter({0: 969, 1: 924})
Checking y_train Balancing: Counter({1: 3809, 0: 3764})
[0.78085809 0.77161716 0.76963696 0.77212682 0.77939234]
----------------------------------------------------------
Checking y_test Balancing: Counter({1: 957, 0: 936})
Checking y_train Balancing: Counter({0: 3797, 1: 3776})
[0.78613861 0.76171617 0.77821782 0.76882431 0.77939234]
----------------------------------------------------------
Checking y_test Balancing: Counter({0: 947, 1: 946})
Checking y_train Balancing: Counter({1: 3787, 0: 3786})
[0.77029703 0.78415842 0.77887789 0.78005284 0.75495376]
----------------------------------------------------------
Checking y_test Balancing: Counter({1: 953, 0: 940})
Checking y_train Balancing: Counter({0: 3793, 1

In [16]:
# 0: 955, 1: 42  --> 94.82
# 0: 944, 1: 52  --> 95.08
# 0: 954, 1: 42  --> 94.83
# 0: 935, 1: 61  --> 95.30
# 0: 945, 1: 51 --> 95.05

### StratifiedKFold Cross Validation

In [75]:
from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold(n_splits=5,random_state=100,shuffle=True)
skf.get_n_splits(x_data,y_data)
print(skf)

StratifiedKFold(n_splits=5, random_state=100, shuffle=True)


In [93]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
l1=LogisticRegression()

In [94]:
scores=cross_val_score(l1,x_train,y_train,cv=skf)
y_pred=cross_val_predict(l1,x_test,y_test,cv=skf)
print(scores)
print(y_pred)

[0.76567657 0.76633663 0.78745875 0.78533686 0.78137384]
[1 1 1 ... 1 1 1]


In [95]:
kx_train_data=[]
kx_test_data=[]
ky_train_data=[]
ky_test_data=[]
for train_data,test_data in skf.split(x_data,y_data):
    x1_train,x1_test=x_data[train_data],x_data[test_data]
    y1_train,y1_test=y_data[train_data],y_data[test_data]
    scores=cross_val_score(l1,x_train,y_train,cv=skf)
    y_pred=cross_val_predict(l1,x_test,y_test,cv=skf)
    kx_train_data.append(x_train)
    kx_test_data.append(x_test)
    ky_train_data.append(y_train)
    ky_test_data.append(y_test)
    print("Checking Balancing.......:",Counter(y_test))
    print(y_pred)
    print(scores)
    print(np.mean(scores)*100)

Checking Balancing.......: Counter({0: 947, 1: 946})
[1 1 1 ... 1 1 1]
[0.76567657 0.76633663 0.78745875 0.78533686 0.78137384]
77.72365294653639
Checking Balancing.......: Counter({0: 947, 1: 946})
[1 1 1 ... 1 1 1]
[0.76567657 0.76633663 0.78745875 0.78533686 0.78137384]
77.72365294653639
Checking Balancing.......: Counter({0: 947, 1: 946})
[1 1 1 ... 1 1 1]
[0.76567657 0.76633663 0.78745875 0.78533686 0.78137384]
77.72365294653639
Checking Balancing.......: Counter({0: 947, 1: 946})
[1 1 1 ... 1 1 1]
[0.76567657 0.76633663 0.78745875 0.78533686 0.78137384]
77.72365294653639
Checking Balancing.......: Counter({0: 947, 1: 946})
[1 1 1 ... 1 1 1]
[0.76567657 0.76633663 0.78745875 0.78533686 0.78137384]
77.72365294653639


In [52]:
d1=pd.Series.to_frame(pd.Series(kx_test_data))
d2=pd.Series.to_frame(pd.Series(ky_test_data))

In [104]:
for train_data,test_data in skf.split(x_data,y_data):
    x_train,x_test=x_data[train_data],x_data[test_data]
    y_train,y_test=y_data[train_data],y_data[test_data]
    scores=cross_val_score(l1,x_train,y_train,cv=skf)
    y_pred=cross_val_predict(l1,x_test,y_test,cv=skf)
    print("Checking y_test Balancing:",Counter(y_test))
    print("Checking y_train Balancing:",Counter(y_train))
    print(scores)
    print("----------------------------------------------------------")

Checking y_test Balancing: Counter({1: 947, 0: 947})
Checking y_train Balancing: Counter({1: 3786, 0: 3786})
[0.77887789 0.77359736 0.77542933 0.77476882 0.78203435]
----------------------------------------------------
Checking y_test Balancing: Counter({1: 947, 0: 946})
Checking y_train Balancing: Counter({0: 3787, 1: 3786})
[0.79273927 0.78283828 0.76039604 0.76882431 0.76155878]
----------------------------------------------------
Checking y_test Balancing: Counter({1: 947, 0: 946})
Checking y_train Balancing: Counter({0: 3787, 1: 3786})
[0.78151815 0.79207921 0.79009901 0.75033025 0.76155878]
----------------------------------------------------
Checking y_test Balancing: Counter({0: 947, 1: 946})
Checking y_train Balancing: Counter({1: 3787, 0: 3786})
[0.76171617 0.76369637 0.78481848 0.7681638  0.79656539]
----------------------------------------------------
Checking y_test Balancing: Counter({0: 947, 1: 946})
Checking y_train Balancing: Counter({1: 3787, 0: 3786})
[0.76567657 0.7

In [111]:
from sklearn.model_selection import ShuffleSplit
sf = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
sf.get_n_splits(x_data,y_data)

5

In [113]:
for train_data,test_data in sf.split(x_data,y_data):
    x_train,x_test=x_data[train_data],x_data[test_data]
    y_train,y_test=y_data[train_data],y_data[test_data]
    scores=cross_val_score(l1, x_train, y_train, cv=sf)
    y_pred=cross_val_predict(l1,x_test,y_test,cv=skf)
    print("Checking y_test Balancing:",Counter(y_test))
    print("Checking y_train Balancing:",Counter(y_train))
    print(np.mean(scores)*100)
    print(scores)
    print("----------------------------------------------")
#     y_pred=cross_val_predict(l1,x_test,y_test,cv=sf)
#     print(Counter(y_test))

Checking y_test Balancing: Counter({1: 950, 0: 944})
Checking y_train Balancing: Counter({0: 3789, 1: 3783})
77.61056105610561
[0.7709571  0.79207921 0.77557756 0.77293729 0.7689769 ]
----------------------------------------------
Checking y_test Balancing: Counter({0: 954, 1: 940})
Checking y_train Balancing: Counter({1: 3793, 0: 3779})
76.14521452145215
[0.77623762 0.75643564 0.76435644 0.75709571 0.75313531]
----------------------------------------------
Checking y_test Balancing: Counter({0: 962, 1: 932})
Checking y_train Balancing: Counter({1: 3801, 0: 3771})
77.62376237623762
[0.7709571  0.78217822 0.77755776 0.77623762 0.77425743]
----------------------------------------------
Checking y_test Balancing: Counter({0: 988, 1: 906})
Checking y_train Balancing: Counter({1: 3827, 0: 3745})
78.21782178217822
[0.79009901 0.78481848 0.76765677 0.78283828 0.78547855]
----------------------------------------------
Checking y_test Balancing: Counter({0: 968, 1: 926})
Checking y_train Balanc

In [86]:
kf.split(x)

<generator object _BaseKFold.split at 0x000001A1D151C270>

In [88]:
skf.split(x,y)

<generator object _BaseKFold.split at 0x000001A1D151C120>